<a href="https://colab.research.google.com/github/Brotherswords/DeepLearning_Project/blob/main/CSCI4931_WUN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Brian H., Jake M., Lavan V., Matthew W.
# CSCI 4931 - Deep Learning Final Project
# Wave-U-Net

In [ ]:
# Environment
!pip install librosa tensorflow
!pip install tensorflow

In [ ]:
!pip install tensorflow

In [ ]:
# Wave-U-Net Model
import tensorflow as tf
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv1D
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/UpSampling1D
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Concatenate
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Normalization
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPooling1D, UpSampling1D, Concatenate, Normalization, BatchNormalization

In [ ]:
print(tf.__version__)

2.12.0


In [ ]:
from keras.layers.normalization import batch_normalization
from tensorflow.python.ops.gen_nn_ops import BatchNormWithGlobalNormalizationGrad
# Wave-U-Net: 1D Audio Processing
# Based on U-Net, using downsampling and upsampling blocks

def BasicWaveUNet(input_shape=(None, 1), num_filters=[16, 32, 64, 128, 256, 512], filter_length=15):

    #input shape parameters
    baych_size = None
    inputs = tf.keras.layers.Input(shape=input_shape)

    # Downsample Layers
    skips = []
    x = inputs
    for filters in num_filters:
        x = Conv1D(filters, filter_length, activation='relu', padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(0.5)(x)
        skips.append(x)
        x = MaxPooling1D(2)(x)

    # Bottleneck Layer
    x = Conv1D(num_filters[-1], filter_length, activation='relu', padding='same')(x)

    # Upsample Layer
    for filters in reversed(num_filters[:-1]):
        x = UpSampling1D(2)(x)
        skip = skips.pop()
        x = Concatenate()([x, skip])
        x = Conv1D(filters, filter_length, activation='relu', padding='same')(x)

    outputs = Conv1D(1, filter_length, activation='linear', padding='same')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    return model

# Create Wave-U-Net model
basic_wave_model = BasicWaveUNet()
basic_wave_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, None, 1)]    0           []                               
                                                                                                  
 conv1d_3 (Conv1D)              (None, None, 16)     256         ['input_4[0][0]']                
                                                                                                  
 batch_normalization_1 (BatchNo  (None, None, 16)    64          ['conv1d_3[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 dropout (Dropout)              (None, None, 16)     0           ['batch_normalization_1[0][0]

Worth an investigation possibly: https://www.tensorflow.org/api_docs/python/tf/keras/layers/DepthwiseConv1D